In [2]:
%matplotlib tk
import warnings
warnings.filterwarnings('once') #only get the hyperspy load warning once

import pycrystem as pc
import numpy as np
import hyperspy.api as hs
from matplotlib import pyplot as plt

In [3]:
from pycrystem.indexation_generator import *
from pycrystem.utils.expt_utils import *
from pycrystem import ElectronDiffraction
from pycrystem.indexation_generator import IndexationGenerator#,mapable_radial_matcher

In [4]:
from load_data import dp #see file for sizing etc
dp#.inav[0,6].plot()

A Jupyter Widget

<ElectronDiffraction, title: , dimensions: (2, 2|144, 144)>

In [5]:
from generate_library import structure,rot_list,edc,diff_gen,struc_lib
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.,
                                            representation='euler')

/home/pc494/anaconda3/lib/python3.6/site-packages/tabulate-0.7.7-py3.6.egg/tabulate.py:178: DeprecationWarning: invalid escape sequence \h
/home/pc494/anaconda3/lib/python3.6/site-packages/tabulate-0.7.7-py3.6.egg/tabulate.py:343: DeprecationWarning: invalid escape sequence \[
/home/pc494/anaconda3/lib/python3.6/site-packages/tabulate-0.7.7-py3.6.egg/tabulate.py:1105: DeprecationWarning: invalid escape sequence \e


In [6]:
indexer = IndexationGenerator(dp, library)
match_results = indexer.correlate(n_largest=5)
## Does this actually give correct answers? Potentially not ... great...

A Jupyter Widget

In [7]:
def _convert_to_2D(peak_locations):
    #if np.any(peak_locations[:,2] > 5e-2):
    #    print("This will be a warning in future, but be aware that you may not be within a 2D regime")
    peak_locations = peak_locations[:,0:2]
    return peak_locations

In [8]:
match_results

<MatchingResults, title: , dimensions: (2, 2|5, 5)>

In [9]:
_phase = 'CsPbBr3'

In [10]:
data_peaks = dp.find_peaks('laplacian_of_gaussians',show_progressbar=False)

/home/pc494/anaconda3/lib/python3.6/site-packages/skimage/feature/blob.py:296: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  sigma_list = np.linspace(min_sigma, max_sigma, num_sigma)


A Jupyter Widget

In [11]:
### so you want to be able to map this
def function_version_1(single):
    best_fit = single[0][1:4]
    best_fit_pattern = library[_phase][(best_fit[0],best_fit[1],best_fit[2])]
    peak_locations = best_fit_pattern.coordinates
    peak_locations = _convert_to_2D(best_fit_pattern.coordinates)
    return peak_locations

In [12]:
def function_version_2(single):
    best_fit = single[0][1:4]
    best_fit_pattern = library[_phase][(best_fit[0],best_fit[1],best_fit[2])]
    return best_fit_pattern.as_signal(144,0.1,1.2)

In [13]:
found_peaks = match_results.map(function_version_1,inplace=False,show_progressbar=False)
from pycrystem.diffraction_vectors import DiffractionVectors
found_peaks = DiffractionVectors(found_peaks)
found_peaks.axes_manager.set_signal_dimension(0)

In [14]:
found_peaks

<DiffractionVectors, title: , dimensions: (2, 2|)>

In [15]:
data_peaks

<DiffractionVectors, title: , dimensions: (2, 2|)>

In [16]:
from pycrystem.utils import plot as pcplt

mmx,mmy = pcplt.generate_marker_inputs_from_peaks(found_peaks)
dp.plot(cmap='viridis') 
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=False)

In [ ]:
#dp.inav[0,0].plot()
best_fit = match_results.inav[0,6].data[0][1:4] 
best_fit_pattern = library['CsPbBr3'][(best_fit[0],best_fit[1],best_fit[2])]

In [ ]:
best_fit_pattern.as_signal(144,0.03,1.2).plot()

<h1> PART II - Speeding up the process using radial forms of the data</h1>

In [28]:
radial_averaged_data = dp.map(radial_average,center=np.asarray([144/2,144/2]),cython=False,inplace=False, show_progressbar=False)
## is this using the correct center

In [ ]:
### This is being rewritten. But should do via Silicon
radial_sims = {}
for XZX in library['CsPbBr3']:
    peaks_2D_data = library['CsPbBr3'][XZX].as_signal(144,0.3,2.1).data
    radial_sims[XZX] = radial_average(peaks_2D_data,center=[144/2,144/2],cython=False)

In [ ]:
%%time
alpha = radial_averaged_data.map(mapable_radial_matcher,dict_of_radial_patterns=radial_sims,n_scorer=10,inplace=False)